## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.
2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).
3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.
Następnie znaleźć minimum - `np.argmin`.
4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.
6. Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

fileName = 'jet.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
img = cv2.imread('jet.bmp', cv2.IMREAD_GRAYSCALE)

plt.imshow(img, 'gray')

In [ ]:
hist = cv2.calcHist([img], [0], None, [256], [0, 256])

fig, ax = plt.subplots()

ax.plot(hist)
ax.set_xlabel("Brightness")
ax.set_ylabel("Amount of pixels")

<h1 style="text-align:center">BBHE</h1>

In [ ]:
mean_brightness = int(cv2.mean(img)[0])
print("Mean brightness value: " + str(mean_brightness))

In [ ]:
h1 = hist[0:169]
h2 = hist[169:]

fig, axs = plt.subplots(1,2)

axs[0].plot(h1)
axs[0].set_xlabel("Brightness")
axs[0].set_ylabel("Amount of pixels")

axs[1].plot([[x] for x in range(169, 256)] , h2)
axs[1].set_xlabel("Brightness")
axs[1].set_ylabel("Amount of pixels")

ymin,ymax = axs[1].get_ylim()

axs[0].set_ylim(ymin,ymax)

In [ ]:
csum1 = np.cumsum(h1)
csum2 = np.cumsum(h2)

fig, axs = plt.subplots(1,2)

axs[0].plot(csum1)
axs[0].set_xlabel("Brightness")
axs[0].set_ylabel("Cumulative amount of pixels")

axs[1].plot([[x] for x in range(169, 256)] , csum2)
axs[1].set_xlabel("Brightness")
axs[1].set_ylabel("Cumulative amount of pixels")

ymin,ymax = axs[1].get_ylim()

axs[0].set_ylim(ymin,ymax)

In [ ]:
for i in range(0, len(csum1)):
    csum1[i] = csum1[i]/(csum1[len(csum1)-1])
for i in range(0, len(csum2)):
    csum2[i] = csum2[i]/(csum2[len(csum2)-1])

fig, axs = plt.subplots(1,2)
    
axs[0].plot(csum1)
axs[0].set_xlabel("Brightness")
axs[0].set_ylabel("Normalized cumulative amount of pixels")

axs[1].plot([[x] for x in range(169, 256)] , csum2)
axs[1].set_xlabel("Brightness")
axs[1].set_ylabel("Normalized cumulative amount of pixels")

ymin,ymax = axs[1].get_ylim()

axs[0].set_ylim(ymin,ymax)

In [ ]:
c1 = csum1.copy()
c2 = csum2.copy()

for i in range(0, len(c1)):
    c1[i] = c1[i]*mean_brightness
for i in range(0, len(c2)):
    c2[i] = c2[i]*(255-mean_brightness-1)+1+mean_brightness
    
fig, axs = plt.subplots(1,2)
    
axs[0].plot(c1)
axs[0].set_xlabel("Brightness")
axs[0].set_ylabel("Remapped brightness")

axs[1].plot([[x] for x in range(169, 256)] , c2)
axs[1].set_xlabel("Brightness")
axs[1].set_ylabel("Remapped brightness")

In [ ]:
joined_hist = np.concatenate((c1, c2))

result = img.copy()
row, col = img.shape
for i in range(row):
        for j in range(col):
            result[i][j] = joined_hist[img[i][j]]
            
plt.imshow(result, 'gray')

In [ ]:
def classic_equalize(img):
    row, col = img.shape
    chist = np.cumsum(cv2.calcHist([img], [0], None, [256], [0, 256]))
    np.ma.masked_equal(chist, 0)
    np.ma.filled(chist)
    for i in range(0, len(chist)):
        chist[i] = int((chist[i]/(chist[len(chist)-1]))*img.max())
        
    result = img.copy()
    
    for i in range(row):
        for j in range(col):
            result[i][j] = chist[img[i][j]]
    
    return result

In [ ]:
eq_img = classic_equalize(img)
fig, axs = plt.subplots(1, 3)
fig.set_size_inches(20, 10)
axs[2].axis('off')
axs[2].imshow(eq_img, 'gray')
axs[2].set_title("Classic HE")
axs[1].axis('off')
axs[1].imshow(result, 'gray')
axs[1].set_title("BBHE")
axs[0].axis('off')
axs[0].imshow(img, 'gray')
axs[0].set_title("Original image")